In [39]:
import json
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import numpy as np

In [59]:
def get_metrics(y_pred, y_test):
    correct = np.sum(y_pred == np.array(y_test))
    acc = correct/len(y_pred)
    fn = 0
    tp = 0
    for p, l in zip (y_pred, np.array(y_test)):
        if l != 0 and p != 0:
            tp += 1
        if l != 0 and p == 0:
            fn += 1
    
    return acc, (fn / (fn + tp))

In [17]:

# node/edge level
data = [json.loads(line) for line in open('./data/graphs/graphs.jsonl', 'r')]

edges = []
labels = []
graph_index = []
for i, graph in enumerate(data):
    for source, dicts in graph.items():
        for target, features in dicts.items():
            labels.append(features.pop('label'))
            edges.append(list(features.values()))
            graph_index.append(i)

In [35]:
X_train, X_test, y_train, y_test = train_test_split(edges, labels, stratify=labels, random_state=1)
# X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, stratify=y_test, random_state=1, train_size=0.5)

In [36]:
clf = MLPClassifier(random_state=1, max_iter=300,early_stopping=True, n_iter_no_change=5)
clf.fit(X_train, y_train)


MLPClassifier(early_stopping=True, max_iter=300, n_iter_no_change=5,
              random_state=1)

In [60]:
y_pred = clf.predict(X_test)
get_metrics(y_pred, y_test)

(0.9767575675291258, 0.994965034965035)

In [79]:
#graph level
data = [json.loads(line) for line in open('./data/graphs/graphs.jsonl', 'r')]

graphs = []
graph_labels = []

graph_index = []
for i, graph in enumerate(data):
    edges = []
    labels = []
    for source, dicts in graph.items():
        for target, features in dicts.items():
            labels.append(features.pop('label'))
            edges.append(list(features.values()))
            graph_index.append(i)
    graphs.append(edges)
    graph_labels.append(labels)

In [93]:
X_train, X_test, y_train, y_test = train_test_split(graphs, graph_labels, random_state=1)
X_train = [x for y in X_train for x in y]
y_train = [x for y in y_train for x in y]

In [94]:
clf = MLPClassifier(random_state=1, max_iter=300,early_stopping=True, n_iter_no_change=5)
clf.fit(X_train, y_train)

MLPClassifier(early_stopping=True, max_iter=300, n_iter_no_change=5,
              random_state=1)

In [98]:
preds = []
trues = []
for g, l in zip(X_test, y_test):
    g_pred = clf.predict(g)
    g_pred = (np.sum(g_pred) > 0).astype(int)
    preds.append(g_pred)
    
    g_true = (np.sum(np.array(l) > 0)).astype(int)
    trues.append(g_true)

In [99]:
get_metrics(preds, trues)

(0.6012, 0.655982905982906)